1. 어떤 데이터를 사용하지?
2. 임베딩 모델, 챗 모델은 뭘로 쓸지?

In [1]:
from datetime import datetime
formatted_date = datetime.today().strftime("%Y-%m-%d %H:%M:%S")
print("코드 돌린 날짜:", formatted_date)

코드 돌린 날짜: 2025-05-20 15:24:45


In [6]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

## Load Data

In [3]:
FILE_PATH = "data/법제소식_7월호_05_최신입법동향.pdf"

### PDFPlumber

In [4]:
from langchain_community.document_loaders import PDFPlumberLoader

# PDF 문서 로더 인스턴스 생성
loader = PDFPlumberLoader(FILE_PATH)

# 문서 로딩
docs = loader.load()

# 첫 번째 문서 데이터 접근
print(docs[0].page_content[:500])

Legislation Newsletter
최신 입법 트렌드
■ 머리말
인공지능(AI) 관련 국내외 법제 동향
● 2022년 말 챗GPT 등장 이후 생성형 AI1)기술 발전이 급격하게 진행되고 있고, 리걸테크, AI챗봇,
AI헬스케어, AI에듀테크 등 인공지능 기술에 기반한 다양한 서비스들이 하나 둘 등장하며 인공
지능이 우리의 삶에 깊이 스며들고 있다. 언론에서는 인공지능에 기반한 새로운 서비스들이 연일
글. 법제처 미래법제혁신기획단 등장할 것을 예고하고 있고, 기존 생성형AI(GPT-3.5)는 인공지능에게 텍스트(또는 음성)의 단일
형태로 지시를 수행하게 하였으나, 최근 공개된 GPT-4o(오픈AI사)와 아스트라(구글) 등은
텍스트뿐만 아니라, 음성, 이미지 등 다양한 소스로부터 데이터를 수집·처리하여 멀티모달
(multi-modal)2)로 실시간 인간의 감정을 분석해 반응하는 등 범용인공지능(AGI; Artificial
General Intelligence)3) 시대의 도래가 멀


### LlamaParser

* 참고자료: https://wikidocs.net/253718

## Vector Database

* ChromaDB

In [ ]:
## 컬렉션 생성/연결하기

import chromadb
client = chromadb.PersistentClient('chroma/')
collection = client.get_or_create_collection(name="dataset")
collection.peek()

{'ids': [],
 'embeddings': array([], dtype=float64),
 'documents': [],
 'uris': None,
 'data': None,
 'metadatas': [],
 'included': [<IncludeEnum.embeddings: 'embeddings'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [3]:
client.list_collections()

[Collection(name=dataset)]

In [7]:
import uuid
from tqdm import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter
from chromadb.utils import embedding_functions

def insert(content, collection_name):
    openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key = OPENAI_API_KEY,
                model_name="text-embedding-3-large" ## text-embedding-ada-002
            )
    collection = client.get_collection(name=collection_name, embedding_function=openai_ef)

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
    docs = text_splitter.split_documents(content)

    for doc in tqdm(docs):
        uuid_val = uuid.uuid1()
        # print("Inserted documents for ", uuid_val)
        collection.add(ids=[str(uuid_val)], documents=doc.page_content, metadatas = doc.metadata)

In [9]:
insert(docs, 'dataset')

100%|██████████| 105/105 [01:18<00:00,  1.34it/s]


In [ ]:
# 저장된 chunk 개수 확인
collection.count()

105

In [ ]:
# sample 10개만 확인하기
collection.peek()

{'ids': ['673006b6-3543-11f0-8c1d-ae08445dcffc',
  '67c968ec-3543-11f0-8c1d-ae08445dcffc',
  '6812efc6-3543-11f0-8c1d-ae08445dcffc',
  '687b2d34-3543-11f0-8c1d-ae08445dcffc',
  '6911b9f2-3543-11f0-8c1d-ae08445dcffc',
  '698b8f5c-3543-11f0-8c1d-ae08445dcffc',
  '6a17f974-3543-11f0-8c1d-ae08445dcffc',
  '6aada42e-3543-11f0-8c1d-ae08445dcffc',
  '6b48d34a-3543-11f0-8c1d-ae08445dcffc',
  '6be691e8-3543-11f0-8c1d-ae08445dcffc'],
 'embeddings': array([[ 0.00460662, -0.0625558 , -0.0140816 , ..., -0.00034537,
         -0.02328175,  0.01535104],
        [ 0.01115845, -0.04634257, -0.02166876, ..., -0.0089194 ,
         -0.00816077, -0.01347116],
        [ 0.01399807, -0.02853678, -0.00617631, ..., -0.0011408 ,
         -0.01105976,  0.00669933],
        ...,
        [-0.00189815, -0.0360109 ,  0.00129869, ..., -0.01440436,
          0.01026719, -0.01089068],
        [ 0.00460921, -0.01506186, -0.00549938, ...,  0.0114575 ,
          0.01661282,  0.00489592],
        [-0.01224026, -0.02425478, 

## Retrieve

In [1]:
import chromadb
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

client = chromadb.PersistentClient('chroma/')
embedding = OpenAIEmbeddings(model='text-embedding-3-large')  
vectorstore = Chroma(client=client, collection_name="dataset", embedding_function=embedding)

In [2]:
# 방법1
input = "인공지능과 관련된 해외의 입법 사례를 정리해줘"
vectorstore.similarity_search(input)

[Document(metadata={'CreationDate': "D:20240711071402Z00'00'", 'Creator': 'Adobe InDesign 15.1 (Macintosh)', 'ModDate': "D:20240711071402Z00'00'", 'Producer': 'Mac OS X 10.13.6 Quartz PDFContext', 'file_path': 'data/법제소식_7월호_05_최신입법동향.pdf', 'page': 1, 'source': 'data/법제소식_7월호_05_최신입법동향.pdf', 'total_pages': 9}, page_content='Legislation Newsletter\n최근 입법 트렌드\n■ 해외입법사례 ● 그리고 2023년 10월 30일, 바이든 정부는 「안전하고 보안이 보장되며 신뢰할 수 있는 인공지능의\n개발과 사용(Safe, Secure, and Trustworthy Development and Use of Artificial Intelligence)에\n● 최근 미국과 유럽연합(EU) 등 세계 각국은 AI와 관련된 글로벌 표준에 자국의 입장을 더 많이 반영 관한 행정명령(제14110호)7)」을 발령하였는데, 이 행정명령에서는 인공지능이 엄청난 잠재력과'),
 Document(metadata={'CreationDate': "D:20240711071402Z00'00'", 'Creator': 'Adobe InDesign 15.1 (Macintosh)', 'ModDate': "D:20240711071402Z00'00'", 'Producer': 'Mac OS X 10.13.6 Quartz PDFContext', 'file_path': 'data/법제소식_7월호_05_최신입법동향.pdf', 'page': 3, 'source': 'data/법제소식_7월호_05_최신입법동향.pdf', 'total_pages': 9}, page_content='Legislation Newsletter\n최근 입법 트렌드\n구분 인터넷

In [42]:
# 방법2
input = "인공지능과 관련된 해외의 입법 사례를 정리해줘"
vectorstore.similarity_search_with_score(input, k=3)

[(Document(metadata={'CreationDate': "D:20240711071402Z00'00'", 'Creator': 'Adobe InDesign 15.1 (Macintosh)', 'ModDate': "D:20240711071402Z00'00'", 'Producer': 'Mac OS X 10.13.6 Quartz PDFContext', 'file_path': 'data/법제소식_7월호_05_최신입법동향.pdf', 'page': 1, 'source': 'data/법제소식_7월호_05_최신입법동향.pdf', 'total_pages': 9}, page_content='Legislation Newsletter\n최근 입법 트렌드\n■ 해외입법사례 ● 그리고 2023년 10월 30일, 바이든 정부는 「안전하고 보안이 보장되며 신뢰할 수 있는 인공지능의\n개발과 사용(Safe, Secure, and Trustworthy Development and Use of Artificial Intelligence)에\n● 최근 미국과 유럽연합(EU) 등 세계 각국은 AI와 관련된 글로벌 표준에 자국의 입장을 더 많이 반영 관한 행정명령(제14110호)7)」을 발령하였는데, 이 행정명령에서는 인공지능이 엄청난 잠재력과'),
  0.9097715020179749),
 (Document(metadata={'CreationDate': "D:20240711071402Z00'00'", 'Creator': 'Adobe InDesign 15.1 (Macintosh)', 'ModDate': "D:20240711071402Z00'00'", 'Producer': 'Mac OS X 10.13.6 Quartz PDFContext', 'file_path': 'data/법제소식_7월호_05_최신입법동향.pdf', 'page': 3, 'source': 'data/법제소식_7월호_05_최신입법동향.pdf', 'total_pages': 9}, page_content='Legislation News

In [2]:
# 방법3
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

input = "인공지능과 관련된 해외의 입법 사례를 정리해줘"
docs = retriever.invoke(input)
docs

[Document(metadata={'CreationDate': "D:20240711071402Z00'00'", 'Creator': 'Adobe InDesign 15.1 (Macintosh)', 'ModDate': "D:20240711071402Z00'00'", 'Producer': 'Mac OS X 10.13.6 Quartz PDFContext', 'file_path': 'data/법제소식_7월호_05_최신입법동향.pdf', 'page': 1, 'source': 'data/법제소식_7월호_05_최신입법동향.pdf', 'total_pages': 9}, page_content='Legislation Newsletter\n최근 입법 트렌드\n■ 해외입법사례 ● 그리고 2023년 10월 30일, 바이든 정부는 「안전하고 보안이 보장되며 신뢰할 수 있는 인공지능의\n개발과 사용(Safe, Secure, and Trustworthy Development and Use of Artificial Intelligence)에\n● 최근 미국과 유럽연합(EU) 등 세계 각국은 AI와 관련된 글로벌 표준에 자국의 입장을 더 많이 반영 관한 행정명령(제14110호)7)」을 발령하였는데, 이 행정명령에서는 인공지능이 엄청난 잠재력과'),
 Document(metadata={'CreationDate': "D:20240711071402Z00'00'", 'Creator': 'Adobe InDesign 15.1 (Macintosh)', 'ModDate': "D:20240711071402Z00'00'", 'Producer': 'Mac OS X 10.13.6 Quartz PDFContext', 'file_path': 'data/법제소식_7월호_05_최신입법동향.pdf', 'page': 3, 'source': 'data/법제소식_7월호_05_최신입법동향.pdf', 'total_pages': 9}, page_content='Legislation Newsletter\n최근 입법 트렌드\n구분 인터넷

In [5]:
# Maximal Marginal Relevance
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 3})

input = "인공지능과 관련된 해외의 입법 사례를 정리해줘"
docs = retriever.invoke(input)
docs

[Document(metadata={'CreationDate': "D:20240711071402Z00'00'", 'Creator': 'Adobe InDesign 15.1 (Macintosh)', 'ModDate': "D:20240711071402Z00'00'", 'Producer': 'Mac OS X 10.13.6 Quartz PDFContext', 'file_path': 'data/법제소식_7월호_05_최신입법동향.pdf', 'page': 1, 'source': 'data/법제소식_7월호_05_최신입법동향.pdf', 'total_pages': 9}, page_content='Legislation Newsletter\n최근 입법 트렌드\n■ 해외입법사례 ● 그리고 2023년 10월 30일, 바이든 정부는 「안전하고 보안이 보장되며 신뢰할 수 있는 인공지능의\n개발과 사용(Safe, Secure, and Trustworthy Development and Use of Artificial Intelligence)에\n● 최근 미국과 유럽연합(EU) 등 세계 각국은 AI와 관련된 글로벌 표준에 자국의 입장을 더 많이 반영 관한 행정명령(제14110호)7)」을 발령하였는데, 이 행정명령에서는 인공지능이 엄청난 잠재력과'),
 Document(metadata={'CreationDate': "D:20240711071402Z00'00'", 'Creator': 'Adobe InDesign 15.1 (Macintosh)', 'ModDate': "D:20240711071402Z00'00'", 'Producer': 'Mac OS X 10.13.6 Quartz PDFContext', 'file_path': 'data/법제소식_7월호_05_최신입법동향.pdf', 'page': 6, 'source': 'data/법제소식_7월호_05_최신입법동향.pdf', 'total_pages': 9}, page_content='인공지능책임법안 고위험 인공지능에 관한 기본계획 수립, 인공지능 분야에 전

In [40]:
docs[0].page_content

'Legislation Newsletter\n최근 입법 트렌드\n【21대 국회에서 발의된 인공지능 관련 9건 법률안 주요내용】\n인공지능의 공정성·투명성·책임성 확보를 위한 시책 수립·시행 지원 및 관련 연구 등을 효율적으로 수행\n신뢰성 전문위원회\n법률명(대표발의 의원, 발의일) 주요내용 하기 위해 위원회에 인공지능 신뢰성 전문위원회를 둔다.'

## Final Answer

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

llm_4o = ChatOpenAI(model="gpt-4o", temperature=0)

model_prompt = PromptTemplate(
    template="""
        You are an expert who answers the query based on the retrieved data.
        Answer in Korean.
        <Question>: {query}
        <Retrieved data>: {retrieved_data}
    """,
    input_variables=["query", "retrieved_data"],
)

def llm_model(input):
    chain = model_prompt | llm_4o 
    answer = chain.invoke({"query": input, "retrieved_data": docs})
    return answer


In [7]:
llm_model("인공지능과 관련된 해외의 입법 사례를 정리해줘")

AIMessage(content='인공지능과 관련된 해외의 입법 사례로는 미국과 유럽연합(EU)의 사례가 있습니다.\n\n1. **미국**: 2023년 10월 30일, 바이든 정부는 "안전하고 보안이 보장되며 신뢰할 수 있는 인공지능의 개발과 사용"에 관한 행정명령(제14110호)을 발령하였습니다. 이 행정명령은 인공지능의 잠재력을 인정하면서도 안전하고 신뢰할 수 있는 개발과 사용을 강조하고 있습니다.\n\n2. **유럽연합(EU)**: EU는 인공지능법 부속서 III에 따라 고위험 AI 시스템을 규제하고 있습니다. 예를 들어, 투표에 영향을 미치기 위해 사용하는 AI 시스템은 특정 요건을 충족해야 하며, 원격 생체인식 시스템은 특정 목적에 한해 사용이 허용됩니다. 이러한 고위험 시스템은 출시 전에 위험성 관리체계 등 7가지 특정 요건을 준수해야 합니다.\n\n이러한 입법 사례들은 인공지능의 안전한 사용과 개발을 촉진하기 위한 국제적인 노력의 일환으로 볼 수 있습니다.', response_metadata={'token_usage': {'completion_tokens': 257, 'prompt_tokens': 904, 'total_tokens': 1161, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_76544d79cb', 'finish_reason': 'stop', 'logprobs': None}, id='run-e9233ab1-a964-4076-9d80-32f7855a709e-0', usage_metadata={'input_toke

In [48]:
# 하나의 함수로 검색부터 답변까지 

def chatbot(input):
    retriever = vectorstore.as_retriever()
    docs = retriever.invoke(input)
    reference = [docs[i].page_content for i in range(len(docs))]
    
    chain = model_prompt | llm_4o 
    answer = chain.invoke({"query": input, "retrieved_data": reference})
    return print("<<Reference docs>>\n", reference, "\n\n<<Answer>>\n", answer.content)

In [49]:
input = "인공지능과 관련된 해외의 입법 사례를 정리해줘"

chatbot(input)

<<Reference docs>>
 ['Legislation Newsletter\n최근 입법 트렌드\n■ 해외입법사례 ● 그리고 2023년 10월 30일, 바이든 정부는 「안전하고 보안이 보장되며 신뢰할 수 있는 인공지능의\n개발과 사용(Safe, Secure, and Trustworthy Development and Use of Artificial Intelligence)에\n● 최근 미국과 유럽연합(EU) 등 세계 각국은 AI와 관련된 글로벌 표준에 자국의 입장을 더 많이 반영 관한 행정명령(제14110호)7)」을 발령하였는데, 이 행정명령에서는 인공지능이 엄청난 잠재력과', 'Legislation Newsletter\n최근 입법 트렌드\n구분 인터넷 또는 CCTV 영상에서 얼굴 이미지를 무작위적인 방식으로 스크래핑하여 얼굴인식 데이터 - 입학 결정이나 할당에 사용되는 AI시스템\n안면 인식 데이터베이스\n베이스를 생성하거나 확장하는 AI시스템은 금지 - 학습 결과 평가에 사용되는 AI시스템\n교육 및 직업훈련\n- 교육 수준 평가 목적으로 사용되는 AI시스템', '● 이러한 인공지능 기술의 급격한 발달에 대응하여 인공지능을 규범적으로 규율하려는 움직임이\n전 세계에서 활발하게 전개되고 있다. 유럽연합(EU)은 인공지능 기술이 시민의 안전과 권리를\n보호하는 방식으로 사용되는 것을 보장할 수 있도록 인공지능의 규제에 초점을 맞춘 「인공지능법\n(Artificial Intelligence Act)」을 제정하였다. 미국은 AI산업의 선두주자로서 「미국인공지능진흥법\n(Advancing American AI Act)」 제정, 바이든 행정부의 행정명령 등을 통해 인공지능 주도권을', 'Legislation Newsletter\n최신 입법 트렌드\n■ 머리말\n인공지능(AI) 관련 국내외 법제 동향\n● 2022년 말 챗GPT 등장 이후 생성형 AI1)기술 발전이 급격하게 진행되고 있고, 리걸테크, AI챗봇,\nAI헬스케어, AI에듀테크 등 인공지능 기술에 기반한 

In [6]:
chromadb.__version__

'0.5.23'

In [13]:
import os
from dotenv import load_dotenv
import chromadb
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
import chainlit as cl

load_dotenv()

################# Retriever ######################

client = chromadb.PersistentClient('chroma/')
embedding = OpenAIEmbeddings(model='text-embedding-3-large')  
vectorstore = Chroma(client=client, collection_name="dataset", embedding_function=embedding)

def retriever(input):
    retriever = vectorstore.as_retriever()
    docs = retriever.invoke(input)
    reference = [docs[i].page_content for i in range(len(docs))]
    return reference

################# LLM ######################

store = {}

def get_session_history(session_ids):
    if session_ids not in store: 
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  


llm_4o = ChatOpenAI(model="gpt-4o", temperature=0)

model_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert who answers the query based on the retrieved data."
            "Answer in Korean.",
        ),
        # 대화기록용 key 인 chat_history 는 가급적 변경 없이 사용하세요!
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "#Question:\n{question} #Retrieved data: \n{retrieved_data}"), 
    ]
)

def chatbot(input):
    references = retriever(input)
    chain = model_prompt | llm_4o
    
    rag_with_history = RunnableWithMessageHistory(
        chain,
        get_session_history,  # 세션 기록을 가져오는 함수
        input_messages_key="question",  # 사용자의 질문이 템플릿 변수에 들어갈 key
        history_messages_key="chat_history",  # 기록 메시지의 키
    )
    
    answer = rag_with_history.invoke({"question": input, "retrieved_data": references}, config={"configurable": {"session_id": "tmp"}},)
    return answer.content

In [14]:
chatbot('hi')

2025-05-21 20:36:42 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-05-21 20:36:47 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'안녕하세요! 고위험 AI 시스템과 관련된 EU 인공지능법에 대한 정보를 찾으시는 것 같습니다. EU 인공지능법에 따르면, 고위험 AI 시스템의 제공자는 여러 가지 의무를 준수해야 합니다. 예를 들어, 시스템의 적합성을 입증하기 위한 기술 문서 작성, 적합성 선언, 그리고 관할 당국에 시스템 출시를 알리는 등의 절차가 포함됩니다. 또한, 시스템 사용으로 인한 위험성을 인식했을 때 즉시 통지하고, 중대한 사건 발생 시 관련 기관에 신고해야 합니다. 이러한 규정은 AI 시스템의 안전성과 신뢰성을 보장하기 위한 것입니다. 추가적인 정보나 구체적인 질문이 있으시면 말씀해 주세요!'